In [1]:
import numpy as np
import pandas as pd

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

C:\Users\dexte\anaconda3\lib\site-packages\bebi103\viz.py:38: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(
C:\Users\dexte\anaconda3\lib\site-packages\bebi103\stan.py:44: UserWarning: Both pystan and cmdstanpy are importable in this environment. As per the cmdstanpy documentation, this is not advised.
  warnings.warn(


Loading BokehJS ...

In [6]:
pip install stan

  Using cached stan-1.0.tar.gz (717 bytes)
  Running setup.py clean for stan
Failed to build stan
    Running setup.py install for stan: started
    Running setup.py install for stan: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\dexte\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\dexte\\AppData\\Local\\Temp\\pip-install-bzhf75ck\\stan\\setup.py'"'"'; __file__='"'"'C:\\Users\\dexte\\AppData\\Local\\Temp\\pip-install-bzhf75ck\\stan\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\dexte\AppData\Local\Temp\pip-wheel-eaekdsa0'
       cwd: C:\Users\dexte\AppData\Local\Temp\pip-install-bzhf75ck\stan\
  Complete output (5 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\dexte\AppData\Local\Temp\pip-install-bzhf75ck\stan\setup.py", line 13, in <module>
      raise Exception(message)
  Exception: You tried to install "stan". The package for PyStan is named "pystan".
  --------------------------------

In [9]:
import stan #install pystan using pip first
import os
os.chdir('../Pystan')
import model_data as md 
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

#arviz is for visualization of stan output
import arviz as az #install using pip first
import xarray as xr

#---------------------------

#number of sessions to do modelling on
numsessions = 5


fnames = ['BH09_raw-free_S1-5_corrected.xlsx','CH02_corrected.xlsx','NA01_raw_free-choice_S8-18.xlsx',"CH01_corrected.xlsx"]
#change these to the names of the excel files that you want to run modeling on 


#--------------------------

#load in excel files into one df, resetting the session numbers to start at 1
df = md.load_multiple_data(fnames, reset_sessions = True)


#creates lists of subjects run on each task (classic A, classic B, etc.)

#rename MSNs so that the rats on the outcome task don't have "loss" in the MSN
for i in range(len(df)):
    if df.at[i, 'MSN'] == 'LossrGT_A-losscue_v1':
        df.at[i,'MSN'] = 'outcomeRGT_A'
    if df.at[i, 'MSN'] == 'LossrGT_B-losscue_v1':
        df.at[i,'MSN'] = 'outcomeRGT_B'
        
#rename MSNs so that the rats on the random task don't have "loss" in the MSN
for i in range(len(df)):
    if df.at[i,'MSN'] == 'AnarchyrGT_B-losscue_v6':
        df.at[i,'MSN'] = 'RandomRGT_B'
    if df.at[i,'MSN'] == 'AnarchyrGT_A-losscue_v6':
        df.at[i,'MSN'] = 'RandomRGT_A'
        
        
task_list = df.groupby(['MSN'])['Subject'].unique()

#concatenate together the lists of subjects that run the same task (i.e., puts version A 
#and version B together) - based on unique string for each task name

##change between model runs
subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Classic' in task]])

#---------------------------------------------


#extract model data from dataframe
model_data = md.get_model_data(df, numsessions, subs)

#stan code

basic_model = """
  // rGT task -- basic RL model

  data {

    // Metadata
    int N;              // number of subjects
    int ntr;            // total number of trials

    //Data
    int startSubject[ntr];  // subject number at start of each subject sequence
    int startSession[ntr];  // session number at start of each session

    int C[ntr];     // chosen stimulus 1-4
    int R[ntr];     // outcome in number of pellets
    int P[ntr];     // punishment in number of seconds
    int O[ntr];   // chosen option (i.e. P1,P2,P3,P4)

  } // data

  parameters{

    // Group level parameters - beta, etaPos, etaNeg, pscale, pintercept
    vector[4] mu_pr;
    vector<lower=0>[4] sigma;

    // Subject-level parameters (raw; these are unit normal for equal density sampling across each par)
    vector[N] beta_samp;
    vector[N] etaPositive_samp;
    vector[N] etaNegative_samp;
    vector[N] m_samp;

  } // parameters

  transformed parameters {

      // Subject-level parameters (transformed)
      vector<lower=0,upper=100>[N]    beta;
      vector<lower=0,upper=1>[N]     etaPositive;
      vector<lower=0,upper=1>[N]     etaNegative;
      vector[N]    m;

      for (i in 1:N) {
          beta[i]  = Phi_approx( mu_pr[1] + sigma[1] * beta_samp[i] ) * 100;
          etaPositive[i] = Phi_approx( mu_pr[2] + sigma[2] * etaPositive_samp[i] );
          etaNegative[i] = Phi_approx( mu_pr[3] + sigma[3] * etaNegative_samp[i] );
          m[i]  = mu_pr[4] + sigma[4] * m_samp[i];
      }

  } // transformed parameters

  model{

    // Group level priors
    mu_pr ~ normal(0, 1);
    sigma ~ gamma(1, 0.5); //normally 0.5, but changing this will change the strength of the group level regularization

    // Subject-level priors - these are for the sampled parameters
    beta_samp ~ normal(0, 1);
    etaPositive_samp ~ normal(0, 1);
    etaNegative_samp ~ normal(0, 1);
    m_samp ~ normal(0, 1);


    // put this in a false loop (error message at vector...)
    for (d in 1:1) {

      //Initialize
      vector[4] Q;
      real delta;
      int currentSubject;
      int currentSession;

      Q = rep_vector(0, 4);
      delta = 0;
      currentSubject = 0;
      currentSession = 0;

      // Likelihood of all the data
      for (tr in 1:ntr){

        if (startSubject[tr]>0) { // if this is the start of a new subject
          currentSubject = startSubject[tr];

          // reset Q values
          Q = rep_vector(0,4);
        }

        if (startSession[tr]>0) { // if this is the start of a new session
          currentSession = startSession[tr];
        }

          //if a choice was made
          if ( O[tr]>0 ) {

            // likelihood of observed choice
            O[tr] ~ categorical_logit( beta[currentSubject] * Q );

            // if positive outcome
            if ( R[tr]>0 ) {
              delta = R[tr] - Q[O[tr]];
              // learn from the positive outcome
              Q[O[tr]] += etaPositive[currentSubject] * delta;
            }
            // if negative outcome
            if ( P[tr]>0 ) {
              delta = -m[currentSubject]*P[tr] - Q[O[tr]];
              // learn from the negative outcome
              Q[O[tr]] += etaNegative[currentSubject] * delta;
            }

          } // if valid choice

        } // trials
      } // d

    } // model
"""

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [13]:
sm = bebi103.stan.StanModel(model_code=basic_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_638d23b1df4f42fa9a2bd669b93ec713 NOW.


In [14]:
samples = sm.sampling(data=model_data)

This will run an error as there is no log_lik value in the model_code 

In [15]:
waic_results = bebi103.stan.waic(samples, log_likelihood='log_lik')
waic_results

AttributeError: module 'bebi103.stan' has no attribute 'waic'